In [57]:
import os

import pandas as pd
import requests
from pydantic import BaseModel, ConfigDict
from sqlalchemy import and_, insert, not_, select, text, tuple_
from utils.bodhi_models import BodhiWaves, BohdiWavesModel
from utils.bodhi_models import engine as bodhi_engine
from utils.bodhi_models import get_session
from typing import List, Dict, Any, Optional
from utils.sl_models import engine as sl_engine

### Test Pydantic Models

In [2]:
with get_session(bodhi_engine) as db:
    stmt = """select * from wave_forecast limit 1"""
    results = db.execute(stmt).fetchall()

/tmp/ipykernel_90876/62947810.py:3: RemovedIn20Warning: Deprecated API features detected! These feature(s) are not compatible with SQLAlchemy 2.0. To prevent incompatible upgrades prior to updating applications, ensure requirements files are pinned to "sqlalchemy<2.0". Set environment variable SQLALCHEMY_WARN_20=1 to show all deprecation warnings.  Set environment variable SQLALCHEMY_SILENCE_UBER_WARNING=1 to silence this message. (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  results = db.execute(stmt).fetchall()


In [3]:
results

[(4182236, '0101000020E610000000000000000024C00000000000001EC0', -7.5, -10.0, datetime.datetime(2024, 6, 12, 0, 0, tzinfo=datetime.timezone.utc), datetime.timedelta(seconds=75600), datetime.datetime(2024, 6, 12, 21, 0, tzinfo=datetime.timezone.utc), 1.7599999904632568, 7.329999923706055, 133.85000610351562, 1.6200000047683716, 7.119999885559082, 126.06999969482422, 7.349999904632568, 122.31999969482422, 0.6499999761581421, 11.680000305175781, datetime.datetime(2024, 6, 12, 13, 27, 53, 61400, tzinfo=datetime.timezone.utc))]

In [4]:
[BohdiWavesModel.model_validate(entry._asdict()) for entry in results]

[BohdiWavesModel(id=4182236, location='0101000020E610000000000000000024C00000000000001EC0', latitude=-7.5, longitude=-10.0, time=datetime.datetime(2024, 6, 12, 0, 0, tzinfo=datetime.timezone.utc), step=datetime.timedelta(seconds=75600), valid_time=datetime.datetime(2024, 6, 12, 21, 0, tzinfo=datetime.timezone.utc), swh=1.7599999904632568, perpw=7.329999923706055, dirpw=133.85000610351562, shww=1.6200000047683716, mpww=7.119999885559082, wvdir=126.06999969482422, ws=7.349999904632568, wdir=122.31999969482422, swell=0.6499999761581421, swper=11.680000305175781, entry_updated=datetime.datetime(2024, 6, 12, 13, 27, 53, 61400, tzinfo=datetime.timezone.utc))]

## Sl Spots by id, lat, and lon

In [5]:
class SpotSpatialIdx(BaseModel):
    spot_id: str
    spot_lat: float
    spot_lon: float

    model_config = ConfigDict(from_attributes=True)

In [6]:
with get_session(sl_engine) as db:
    stmt = "select spot_id, spot_lat, spot_lon from sl_spots"
    results = db.execute(stmt).fetchall()


In [7]:
spatial_idxs = [SpotSpatialIdx.model_validate(entry) for entry in results]

### Get the associated Offshore Location for each spot

In [8]:
with get_session(sl_engine) as db:
    stmt = text("""select distinct on ("associated_spotId") "associated_spotId", "associated_offshoreLocation_lat", "associated_offshoreLocation_lon" from sl_ratings""")
    results = db.execute(stmt).fetchall()

In [9]:
class SlOffshoreIdx(BaseModel):
    associated_spotId: str
    associated_offshoreLocation_lat: float
    associated_offshoreLocation_lon: float

    model_config = ConfigDict(from_attributes=True)

In [10]:
data = [SlOffshoreIdx.model_validate(entry) for entry in results]

In [11]:
data_dicts = [SlOffshoreIdx.model_dump(entry) for entry in data]

#### Transform to dataframe

Filter to only include where the spot's offshore location matches bodhi-cast's offshore location

In [12]:
df = pd.DataFrame(data_dicts)

In [13]:
df.head()

,associated_spotId,associated_offshoreLocation_lat,associated_offshoreLocation_lon
0,5842041f4e65fad6a77087f9,37.50,-122.75
1,5842041f4e65fad6a7708804,45.25,-124.25
2,5842041f4e65fad6a7708805,36.75,-122.25
3,5842041f4e65fad6a7708806,36.75,-122.25
4,5842041f4e65fad6a7708807,36.90,-122.10


Create a mask to only keep lat an lon where they are in the intervals .0, .25, .5, .75

In [14]:
df['lat_mod'] = df['associated_offshoreLocation_lat'] % 4
df['lon_mod'] = df['associated_offshoreLocation_lon'] % 4

In [15]:
mask = (df['lat_mod'].apply(lambda x: round(x, 2) == x) & df['lon_mod'].apply(lambda x: round(x, 2) == x))

In [16]:
df = df[mask]

In [17]:
df = df.drop(columns=['lat_mod', 'lon_mod'])

In [18]:
len(df)

579

In [19]:
lat_lon_list = list(zip(df['associated_offshoreLocation_lat'].values, df['associated_offshoreLocation_lon'].values))

In [20]:
lat_lon_str = ', '.join(map(str, lat_lon_list))

### Enable postgis

In [21]:
with get_session(sl_engine) as db:
    stmt = text("""CREATE EXTENSION IF NOT EXISTS postgis""")
    results = db.execute(stmt)
    db.commit()

### Create indexes and reindex 

In [22]:
with get_session(bodhi_engine) as db:
    stmt = text("""CREATE INDEX if not exists idx_wave_forecast_lat_lon ON wave_forecast (latitude, longitude)""")
    results = db.execute(stmt)
    db.commit()

In [23]:
with get_session(bodhi_engine) as db:
    stmt = text("""reindex index idx_wave_forecast_lat_lon""")
    results = db.execute(stmt)
    db.commit()

### Getting matching bodhi wave data

Get all waves from bodhi for the current day that match the filtered sl spots 

In [24]:
with get_session(bodhi_engine) as db:
    stmt = text(f"""select * from wave_forecast where time = CURRENT_DATE AND (latitude, longitude) in ({lat_lon_str}) limit 5""")
    results = db.execute(stmt).fetchall()

In [25]:
data = [BohdiWavesModel.model_validate(entry) for entry in results]

In [26]:
data

[BohdiWavesModel(id=30254, location='0101000020E610000000000000008061C00000000000C04D40', latitude=59.5, longitude=-140.0, time=datetime.datetime(2024, 6, 12, 0, 0, tzinfo=datetime.timezone.utc), step=datetime.timedelta(0), valid_time=datetime.datetime(2024, 6, 12, 0, 0, tzinfo=datetime.timezone.utc), swh=1.9600000381469727, perpw=10.489999771118164, dirpw=218.67999267578125, shww=None, mpww=None, wvdir=None, ws=4.110000133514404, wdir=269.8800048828125, swell=1.8700000047683716, swper=10.489999771118164, entry_updated=datetime.datetime(2024, 6, 12, 13, 19, 52, 744570, tzinfo=datetime.timezone.utc)),
 BohdiWavesModel(id=34323, location='0101000020E610000000000000000063C00000000000E04C40', latitude=57.75, longitude=-152.0, time=datetime.datetime(2024, 6, 12, 0, 0, tzinfo=datetime.timezone.utc), step=datetime.timedelta(0), valid_time=datetime.datetime(2024, 6, 12, 0, 0, tzinfo=datetime.timezone.utc), swh=1.3300000429153442, perpw=8.25, dirpw=162.5, shww=0.009999999776482582, mpww=0.20000

## Create New Table to push to same postgres db as other sl data

In [27]:
from utils.bodhi_models import create_tables as create_bodhi_tables


In [28]:
create_bodhi_tables(sl_engine)

In [39]:
data_dicts = [entry.model_dump() for entry in data]

In [40]:
for d in data_dicts:
    d.pop("location", None)

In [41]:
data_dicts[0].keys()

dict_keys(['id', 'latitude', 'longitude', 'time', 'step', 'valid_time', 'swh', 'perpw', 'dirpw', 'shww', 'mpww', 'wvdir', 'ws', 'wdir', 'swell', 'swper', 'entry_updated'])

In [42]:
with get_session(sl_engine) as db:
    stmt = insert(BodhiWaves).values(data_dicts)
    db.execute(stmt)
    db.commit()

In [55]:
def fetch_wave_data(bs: int = 10):
    with get_session(bodhi_engine) as db:
        stmt = text(
            f"""select * from wave_forecast where time = CURRENT_DATE AND (latitude, longitude) in ({lat_lon_str}) limit {bs}"""
        )
        results = db.execute(stmt).fetchall()
        data = [BohdiWavesModel.model_validate(entry) for entry in results]
        data_dict = [entry.model_dump() for entry in data]
        for d in data_dict:
            d.pop("location", None)
    return data_dict

In [58]:
def wave_data_to_db(data: List[Dict], bs: int = 10):
    with get_session(sl_engine) as db:
        stmt = insert(BodhiWaves).values(data)
        db.execute(stmt)
        db.commit()


SyntaxError: non-default argument follows default argument (2102384898.py, line 1)

In [56]:
fetch_wave_data(1)

[{'id': 30254,
  'latitude': 59.5,
  'longitude': -140.0,
  'time': datetime.datetime(2024, 6, 12, 0, 0, tzinfo=datetime.timezone.utc),
  'step': datetime.timedelta(0),
  'valid_time': datetime.datetime(2024, 6, 12, 0, 0, tzinfo=datetime.timezone.utc),
  'swh': 1.9600000381469727,
  'perpw': 10.489999771118164,
  'dirpw': 218.67999267578125,
  'shww': None,
  'mpww': None,
  'wvdir': None,
  'ws': 4.110000133514404,
  'wdir': 269.8800048828125,
  'swell': 1.8700000047683716,
  'swper': 10.489999771118164,
  'entry_updated': datetime.datetime(2024, 6, 12, 13, 19, 52, 744570, tzinfo=datetime.timezone.utc)}]